# Making a count-based distributional model


## A tiny space

We start with a tiny corpus, so that we can easily inspect our data: 

In [1]:
sam_corpus = """I am Sam.
Sam I am.
I do not like green eggs and ham."""


Here is how to split the corpus into sentences using the Natural Language Toolkit:

In [2]:
import nltk

sam_sentences = nltk.sent_tokenize(sam_corpus)
sam_sentences

['I am Sam.', 'Sam I am.', 'I do not like green eggs and ham.']

We need to get the context around a target word. To do this, we make a function that, for a target index, yields the words preceding and succeeding (if any). For simplicity, we use a one-word window on either side of the target. 

(It has "yield" instead of "return", so it is intended to be used in a for-loop.)

In [ ]:
def each_contextword_1wordwindow(wordlist, targetindex):
    if targetindex > 0:
        # preceding word
        yield wordlist[targetindex - 1]
        
    if targetindex < len(wordlist)- 1:
        # succeeding word
        yield wordlist[targetindex + 1]


def each_contextword_2wordwindow(wordlist, targetindex):
    if targetindex > 1:
        # preceding word 1
        yield wordlist[targetindex - 1]
        # preceding word 2
        yield wordlist[targetindex - 2]
    
    if targetindex < len(wordlist) - 2:
        # succeeding word 1
        yield wordlist[targetindex + 1]
        # succeeding word 2
        yield wordlist[targetindex + 2]

        

Here is how to use it: We count, for each target word, how often each context word appears with it. 

If you wanted to use a larger context, you could extend the function. For a context window of 2 words on either side (but not crossing sentence boundaries), for example, you would add two more `yield` statements to the function so that it also yields the word 2 indices before the target, and the word 2 indices after the target.

## An aside: data structures for counting words

As an aside, first a few quick words about data structures in NLTK that support us in counting words (or word groups, or pieces of syntactic structure). The first is basically a dictionary mapping words to counts, called a FreqDist. Conveniently, you can just initialize it by giving it a list of items, and it will count how often each item appears in the list:

In [4]:
# NLTK data structures for counting stuff:
# count individual words or other items:

fd = nltk.FreqDist(["a", "b", "c", "a", "b", "d"])
fd

FreqDist({'a': 2, 'b': 2, 'c': 1, 'd': 1})

The second data structure relevant for us today is the ConditionalFreqDist. It also has counts, but it can be used to count, for each target, how often each context word appears, or more generally, how often each word appears given some other word. Say "a" is a target, and "b" and "c" are context items, then a ConditionalFreqDist can be used like a two-deep dictionary, whose first-level keys are called "conditions":

In [5]:
# for targets, count context words,
# or in general, for one sort of items, 
# count another sort of items
cfd = nltk.ConditionalFreqDist()
cfd["a"]["b"] += 1
cfd["a"]["c"] += 1
cfd

<ConditionalFreqDist with 1 conditions>

For the "condition" 'a', the entry is again a FreqDist object that counts appearances of 'b' and 'c':

In [6]:
cfd["a"]

FreqDist({'b': 1, 'c': 1})

You can also initialize a ConditionalFreqDist by a list of pairs. It then counts, for each first item of the pair, how often each second item appears. In the next example, the ConditionalFreqDist will record that given "a", both "b" and "c" appeared once, and that given "d", "e" appeared once:

In [7]:
cfd = nltk.ConditionalFreqDist([("a", "b"), ("a", "c"), ("d", "e")])
cfd

<ConditionalFreqDist with 2 conditions>

In [8]:
cfd["a"]

FreqDist({'b': 1, 'c': 1})

## Counting words in the "Sam corpus"

Now back to our Sam corpus. We can count context words for each target using a ConditionalFreqDist where the conditions are targets, and the keys of the FreqDist's are context words:

In [9]:
# here we will store the target words and their context word counts
# this is a data type with method  conditions() to get the list of target words,
# and sam_context_counts[t][cx] gets you the count for target t and context word cx
sam_context_counts = nltk.ConditionalFreqDist()

# iterate
for sentence in sam_sentences:
    wordlist = nltk.word_tokenize(sentence)
    
    for targetindex, target in enumerate(wordlist):
        for contextword in each_contextword_1wordwindow(wordlist, targetindex):
            sam_context_counts[target][contextword] += 1


In [10]:
# Here are the target words from our corpus
sam_context_counts.conditions()

['I', 'am', 'Sam', '.', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham']

In [11]:
# Here are the counts for one target
sam_context_counts["eggs"]

FreqDist({'green': 1, 'and': 1})

Here is how to read this: For the target word "eggs", we've seen the word "green" once in its context, and the word "and" once in its context.

Here are the context counts for all the targets in the corpus. The output shows first the target word, then the context words and their counts as a FreqDist object.

In [12]:
list(sam_context_counts.items())

[('I', FreqDist({'am': 2, 'Sam': 1, 'do': 1})),
 ('am', FreqDist({'I': 2, 'Sam': 1, '.': 1})),
 ('Sam', FreqDist({'am': 1, '.': 1, 'I': 1})),
 ('.', FreqDist({'Sam': 1, 'am': 1, 'ham': 1})),
 ('do', FreqDist({'I': 1, 'not': 1})),
 ('not', FreqDist({'do': 1, 'like': 1})),
 ('like', FreqDist({'not': 1, 'green': 1})),
 ('green', FreqDist({'like': 1, 'eggs': 1})),
 ('eggs', FreqDist({'green': 1, 'and': 1})),
 ('and', FreqDist({'eggs': 1, 'ham': 1})),
 ('ham', FreqDist({'and': 1, '.': 1}))]

In [13]:
# Here's an example of how to access one count 
# for target "I" and context "do"
sam_context_counts["I"]["do"]

1

In [14]:
# We put our rows and columns in order.
# For now, our rows are the same as our columns:
# All target words are also context words, and vice versa.
# But that doesn't have to be the case.
targetlist = sorted(sam_context_counts.conditions())
contextlist = sorted(list(set(c for t in sam_context_counts.conditions() 
                              for c in sam_context_counts[t].keys())))

print("Targets", targetlist)
print("Contexts", contextlist)

Targets ['.', 'I', 'Sam', 'am', 'and', 'do', 'eggs', 'green', 'ham', 'like', 'not']
Contexts ['.', 'I', 'Sam', 'am', 'and', 'do', 'eggs', 'green', 'ham', 'like', 'not']


## Working with numpy

numpy is a Python package for working with vectors and matrices. Here is a vector:

In [15]:
import numpy as np

a = np.array([1,2,3])
a

array([1, 2, 3])

In [16]:
# we can address entries in the vector by their index
a[0]

1

In [17]:
# and here is a two-dimensional matrix
m = np.array([[1,2,3], [4,5,6]])
m

array([[1, 2, 3],
       [4, 5, 6]])

In [18]:
# let's address the 3rd entry in the 2nd row.
# here we use *two* indices separated by comma
m[1,2]

6

Now we put all our counts into a numpy matrix.

In [19]:
sam_rows = [ ]
for t in targetlist:
    # for context words c for which we don't have an entry,
    # the ConditionalFreqDist returns zero
    sam_rows.append( [sam_context_counts[t][c] for c in contextlist])
    
sam_count_matrix = np.array(sam_rows)
sam_count_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]])

But which row goes with which target, and which column goes with which context word? Let's make a lookup dictionary for this. We make separate dictionaries for targets and contexts; even though they are the same in our cases, that needn't always be so.

In [20]:
sam_target_dict = { }
for index, target in enumerate(targetlist):
    sam_target_dict[target] = index
    
sam_context_dict = { }
for index, context in enumerate(contextlist):
    sam_context_dict[context] = index

In [21]:
# now we can use it like this:
print("All targets:", targetlist)
print("row for Sam:", sam_target_dict["Sam"])
#notation: we only give *one* index here, to select the entire row
print('and the row is:', sam_count_matrix[ sam_target_dict["Sam"]])

All targets: ['.', 'I', 'Sam', 'am', 'and', 'do', 'eggs', 'green', 'ham', 'like', 'not']
row for Sam: 2
and the row is: [1 1 0 1 0 0 0 0 0 0 0]


In [22]:
print("All contexts:", contextlist)
print("column for am:", sam_context_dict["am"])
# look at the notation: the first index is :, for "all rows".
# the second index is the column we want
print("and the column is:", sam_count_matrix[:, sam_context_dict["am"]])

All contexts: ['.', 'I', 'Sam', 'am', 'and', 'do', 'eggs', 'green', 'ham', 'like', 'not']
column for am: 3
and the column is: [1 2 1 0 0 0 0 0 0 0 0]


## Cosine similarity

Now we can compute cosine similarity in our space. Python has a built-in function for this -- with one catch: It computes 1 - cosine, as a distance. Here's how to get back to cosine similarity:

In [23]:
import scipy

def cosine_sim(vec1, vec2):
    return 1 - scipy.spatial.distance.cosine(vec1, vec2)

cosine_sim(sam_count_matrix[ sam_target_dict["I"]], 
           sam_count_matrix[ sam_target_dict["Sam"]])



0.4714045207910318

If we want to build our own cosine similarity function, here's how to do this. I've made the implementation quite verbose so you can see what is what.

In [24]:
import math

def ourcosine_sim(vec1, vec2):
    # for the denominator: vector lengths
    veclen1 = math.sqrt(sum(vec1**2))
    veclen2 = math.sqrt(sum(vec2**2))
    
    # for the numerator: dot product
    dotprod =  sum(vec1 * vec2)
    
    return dotprod / (veclen1 * veclen2)

In [25]:
ourcosine_sim(sam_count_matrix[ sam_target_dict["I"]], 
              sam_count_matrix[ sam_target_dict["Sam"]])

0.47140452079103173

## From word co-occurrence counts to association weights

As we discussed in class, raw frequency counts may not be what we want -- we don't need to know that all words co-occur a lot with "the" and "a". Even if we ditch stopwords, the frequency bias in the data may not be what we want: Do we need to know that all words co-occur a lot with "said"? 

Several methods have been developed for going from counts to association weights, including tf/idf and pointwise mutual information. Here, we demonstrate how to compute pointwise mutual information, defined as

$PMI(a, b) = \log \frac{P(a, b)}{P(a)P(b)}$ 

In the numerator, we have the joint probability of a *and* b. The formula compares this to the denominator, which has the product of the probability of a and the probability of b: If a and b were completely independent, had zero association, we would expect them to co-occur only by chance, that is, we would expect $P(a, b) = P(a)P(b)$. If $P(a, b)$ is larger than $P(a)P(b)$, then a and b are positively associated -- they co-occur more often than you would expect just from chance encounters. If $P(a, b)$ is smaller than $P(a)P(b)$, then a and b are negatively associated -- they really don't want to go together. 

In practice, we are often not interested in negative associations, and only use positive ones. Then we get PPMI:


$PPMI(a, b) = \left\{\begin{array}{ll}PMI(a, b) & \text{if } PMI(a, b) > 0\\
0 & \text{else}
\end{array}\right.$



In [26]:
# Here are the pieces we need:
# pointwise mutual information (PMI):
#                    P(t, c)
# PMI(t, c) = log --------------
#                   P(t) P(c)
#
#    #(t, c): the co-occurrence count of t with c
#    #(_, _): the sum of counts in the whole table, across all targets
#    #(t, _): the sum of counts in the row of target t
#    #(_, c): the sum of counts in the column of context item c
#
# then
# P(t, c) = #(t, c) / #(_, _)
# P(t) = #(t, _) / #(_, _)
# P(c) = #(_, c) / #(_, _)
#
# PPMI(t, c) = { PMI(t, c) if PMI(t, c) >= 0
#                0, else

# target count #(t, _):
print("target count for Sam", sam_count_matrix[ sam_target_dict["Sam"]].sum())

# overall count #(_, _):
print("overall count", sam_count_matrix.sum())

# context item count #(_, c):
print("context item count for eggs", sam_count_matrix[ :, sam_context_dict["eggs"]].sum())

target count for Sam 3
overall count 28
context item count for eggs 2


In [27]:
import math
from collections import defaultdict

# we'll store the association weights in a dictionary for now
sam_pmi = { }

count_all = sam_count_matrix.sum()

for target in targetlist:
    for context in contextlist:
        p_t_c = sam_count_matrix[ sam_target_dict[target], sam_context_dict[context]] / count_all
        # print("p_t_c", target, context, p_t_c)
        p_t = sam_count_matrix[sam_target_dict[target]].sum() / count_all
        # print("p_t", target, p_t)
        p_c = sam_count_matrix[:, sam_context_dict[context]].sum() / count_all
        # print("p_c", context, p_c)

        # we need to watch out: if p_t_c is zero, the logarithm is undefined
        if p_t_c == 0.0 or p_t == 0.0 or p_c == 0.0:
            pmi = 0.0
        else: 
            pmi = math.log( p_t_c / (p_t * p_c))
        
        # print("pmi", target, context, pmi)
        
        if target not in sam_pmi: 
            # this is a dictionary that, where we haven't set a value,
            # returns 0.0
            sam_pmi[ target] = defaultdict(float)
        sam_pmi[target][context]= pmi


In [28]:
# And we again store the result in a matrix

sam_rows = [ ]
for t in targetlist:
    # for context words c for which we don't have an entry,
    # the ConditionalFreqDist returns zero
    sam_rows.append( [sam_pmi[t][c] for c in contextlist])
    
sam_pmi_matrix = np.array(sam_rows)
# let's print the matrix with only 2 decimal points
# to make it look prettier
sam_pmi_matrix.round(2)


array([[0.  , 0.  , 1.13, 0.85, 0.  , 0.  , 0.  , 0.  , 1.54, 0.  , 0.  ],
       [0.  , 0.  , 0.85, 1.25, 0.  , 1.25, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.13, 0.85, 0.  , 0.85, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.85, 1.25, 0.85, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.95, 0.  , 1.95, 0.  , 0.  ],
       [0.  , 1.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.95],
       [0.  , 0.  , 0.  , 0.  , 1.95, 0.  , 0.  , 1.95, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.95, 0.  , 0.  , 1.95, 0.  ],
       [1.54, 0.  , 0.  , 0.  , 1.95, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.95, 0.  , 0.  , 1.95],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.95, 0.  , 0.  , 0.  , 1.95, 0.  ]])

In [29]:
# We can directly compare counts and associations
print("Counts for dot:")
print("\t", sam_count_matrix[sam_target_dict["."]])
print("Associations for dot:")
print("\t", sam_pmi_matrix[sam_target_dict["."]].round(2))


Counts for dot:
	 [0 0 1 1 0 0 0 0 1 0 0]
Associations for dot:
	 [0.   0.   1.13 0.85 0.   0.   0.   0.   1.54 0.   0.  ]


In [30]:
# How has that changed cosines?
print("Cosine of 'I' and 'Sam', count-based", 
      cosine_sim(sam_count_matrix[sam_target_dict["I"]], 
                 sam_count_matrix[sam_target_dict["Sam"]]))
print("Cosine of 'I' and 'Sam', pmi-based", 
      cosine_sim(sam_pmi_matrix[sam_target_dict["I"]], 
                 sam_pmi_matrix[sam_target_dict["Sam"]]))

Cosine of 'I' and 'Sam', count-based 0.4714045207910318
Cosine of 'I' and 'Sam', pmi-based 0.3274843356832646


In [31]:
# Here is how to get all pairwise cosines in our matrix:

# Step 1: this computes pairwise cosine *distances*, 1 - cosine
sam_cosine_dist = scipy.spatial.distance.cdist(sam_pmi_matrix, sam_pmi_matrix, metric = "cosine")

# Step 2: convert to cosine similarity
sam_cosine_sim = 1 - sam_cosine_dist

# Let's look at this. sam_cosine_sim is a numpy ndarray, which
# has a method round() for rounding.
# As you can see, the cosine similarity of each word with itself is 1,
# and the value for "i" and "sam", row 2, column 3, is 0.33, as computed above.
sam_cosine_sim.round(2)

array([[1.  , 0.49, 0.21, 0.27, 0.52, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.49, 1.  , 0.33, 0.21, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.45],
       [0.21, 0.33, 1.  , 0.71, 0.  , 0.28, 0.  , 0.  , 0.43, 0.  , 0.  ],
       [0.27, 0.21, 0.71, 1.  , 0.  , 0.39, 0.  , 0.  , 0.3 , 0.  , 0.  ],
       [0.52, 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.28, 0.39, 0.  , 1.  , 0.  , 0.  , 0.  , 0.59, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.55, 0.5 , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 1.  , 0.  , 0.  , 0.5 ],
       [0.  , 0.  , 0.43, 0.3 , 0.  , 0.  , 0.55, 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.59, 0.5 , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.45, 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 1.  ]])

## Nearest neighbors

We want to know about a word's nearest neighbors. Computing this by hand is a major pain: You would have to compute all pairwise cosines, and then rummage through them to find the maximum. In our tiny Sam corpus, this is feasible, but not in a large corpus. Fortunatly scikit-learn has a function NearestNeighbors that can do the work for us. One downside: It does not know cosine similarity outright. 

First option: We give it the cosine distance function that we used above. 

In [32]:
from sklearn.neighbors import NearestNeighbors

# we make a nearest-neighbors object and tell it we'll always want the 3 nearest neighbors at a time
nearest_neighbors_obj = NearestNeighbors(n_neighbors=3, metric = scipy.spatial.distance.cosine)

# we then allow it to compute an internal datastructure from our data
nearest_neighbors_obj.fit(sam_pmi_matrix)

NearestNeighbors(metric=<function cosine at 0x10d5d1940>, n_neighbors=3)

In [33]:
cosine_distances, target_indices = nearest_neighbors_obj.kneighbors([sam_pmi_matrix[sam_target_dict["Sam"]]])

In [34]:
# cosine_distances and target_indices are both two-dimensional arrays. Let's extract 
# lists of values
cosine_distances = cosine_distances[0].tolist()
target_indices = target_indices[0].tolist()

for cosinedist, targetindex in zip(cosine_distances, target_indices):
    print("Neighbor is", targetlist[targetindex], "with similarity", 1 - cosinedist)

Neighbor is Sam with similarity 1.0
Neighbor is am with similarity 0.707098356669986
Neighbor is ham with similarity 0.42683129808958764


Second option: We use Euclidean distance (walking distance), but first normalize all vectors to be of length one. This won't give us the same distance values, but the same orderings of what is nearest. See https://stackoverflow.com/questions/34144632/using-cosine-distance-with-scikit-learn-kneighborsclassifier

The length of a vector is defined as 

$||a|| = \sqrt{\sum_i a_i^2}$

The Python package numpy has an implementation of that. For a vector `a`, we need to call `numpy.linalg.norm(a, ord = 2)`.

Note: The sum of values of a vector is called its "L1 norm", and the vector length is called its "L2 norm". That's why we need the parameter `ord = 2`.

In [35]:
import numpy as np

rows = [ sam_pmi_matrix[sam_target_dict[t]] / 
        np.linalg.norm(sam_pmi_matrix[sam_target_dict[t]], ord = 2)
        for t in targetlist ]
   
sam_pminorm_matrix = np.array(rows)

sam_pminorm_matrix.round(2)

array([[0.  , 0.  , 0.54, 0.4 , 0.  , 0.  , 0.  , 0.  , 0.74, 0.  , 0.  ],
       [0.  , 0.  , 0.43, 0.64, 0.  , 0.64, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.69, 0.51, 0.  , 0.51, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.49, 0.72, 0.49, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.71, 0.  , 0.  ],
       [0.  , 0.54, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.84],
       [0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.  , 0.71, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.  , 0.71, 0.  ],
       [0.62, 0.  , 0.  , 0.  , 0.78, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.  , 0.71],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.  , 0.  , 0.  , 0.71, 0.  ]])

In [36]:
# we make a nearest-neighbors object 
# and tell it we'll always want the 2 nearest neighbors at a time.
# Distance metric is default: 
# minkowski with p=2, which is equivalent to Euclidean distance
nearest_neighbors_obj_2 = NearestNeighbors(n_neighbors=3)

# we then allow it to compute an internal datastructure from our data
nearest_neighbors_obj_2.fit(sam_pminorm_matrix)

NearestNeighbors(n_neighbors=3)

In [37]:
# and let's look at nearest neighbors again
distances, target_indices = nearest_neighbors_obj_2.kneighbors([sam_pmi_matrix[sam_target_dict["Sam"]]])

# cosine_distances and target_indices are both two-dimensional arrays. Let's extract 
# lists of values
distances = distances[0].tolist()
target_indices = target_indices[0].tolist()

for dist, targetindex in zip(distances, target_indices):
    print("Neighbor is", targetlist[targetindex], "with distance", dist)

Neighbor is Sam with distance 0.6504565357441716
Neighbor is am with distance 1.1789557107969613
Neighbor is ham with distance 1.521536646024799


## Back to pre-computed spaces

The pre-computed spaces that we tested in our previous notebook were offered within `gensim`, and they were in gensim format. But not all pre-computed spaces come as gensim data objects. Some pre-computed spaces are distributed as a matrix of numbers -- just like the ones we computed above for the Sam corpus. This is the case for example for the GloVE vectors from the original GloVe website: https://nlp.stanford.edu/projects/glove/

To work with a space like that, you can use the same code as for the Sam corpus above.


In [38]:
# In case you are fast-forwarding through this notebook, let's quit here
# in case you don't want to run the following more time-consuming
# steps
raise Exception("stopping fast-forward here")

# Using a somewhat larger corpus

We next demonstrate a somewhat larger corpus, with yet another method of accessing the corpus data: If the data is available within the NLTK corpora, you can use the NLTK's corpus reader to access it.

The Brown corpus is a 1 million word corpus of carefully selected text pieces from different genres, originally made to support dictionary-makers, so it's intended to cover a broad variety of genres in English.

In [39]:
print("The first few sentences of the Brown corpus:\n")
for s in nltk.corpus.brown.sents()[:3]: 
    print(s, "\n")

The first few sentences of the Brown corpus:

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'] 

['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'] 

['The', 'September-October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate', 'Ivan', 'Allen', 'Jr.', '.'] 



We compute the target/context counts, noting context items as we go. We only count words that appear at least 10 times in the corpus. This cuts down a lot on the size of our matrix.

In [40]:
brown_wordcounts = nltk.FreqDist(nltk.corpus.brown.words())
brown_wordcounts.most_common(20)

[('the', 62713),
 (',', 58334),
 ('.', 49346),
 ('of', 36080),
 ('and', 27915),
 ('to', 25732),
 ('a', 21881),
 ('in', 19536),
 ('that', 10237),
 ('is', 10011),
 ('was', 9777),
 ('for', 8841),
 ('``', 8837),
 ("''", 8789),
 ('The', 7258),
 ('with', 7012),
 ('it', 6723),
 ('as', 6706),
 ('he', 6566),
 ('his', 6466)]

In [41]:
import string

brown_context_counts = nltk.ConditionalFreqDist()

frequency_threshold = 20

for sentence in nltk.corpus.brown.sents():
    # remove punctuation.
    # at this point you could also remove stopwords
    # or iterate over sents_tagged() instead of sents()
    # to get parts of speech, and only retain
    # content words
    wordlist = [w for w in sentence if w.strip(string.punctuation) != ""]
    
    for targetindex, target in enumerate(wordlist):
        for contextword in each_contextword_1wordwindow(wordlist, targetindex):
            if brown_wordcounts[target] >= frequency_threshold and brown_wordcounts[contextword] >= frequency_threshold:
                brown_context_counts[target][contextword] += 1   


For this larger corpus, it now makes sense to look at some context word counts to get a sense of what the tables of counts tell us. 

In [42]:
# 10 most frequent context words: similar across many items
# (what can we do about that?)
print("10 most frequent contexts for some targets:\n")
print("election:\n", brown_context_counts["election"].most_common(10))
print("love:\n", brown_context_counts["love"].most_common(10))
print("car:", brown_context_counts["car"].most_common(10))

10 most frequent contexts for some targets:

election:
 [('the', 21), ('of', 11), ('to', 6), ('for', 4), ('was', 3), ('and', 3), ('his', 3), ('I', 3), ('on', 3), ('in', 3)]
love:
 [('of', 36), ('and', 34), ('in', 26), ('for', 22), ('to', 21), ('the', 17), ('with', 14), ('I', 13), ('you', 10), ('is', 10)]
car: [('the', 84), ('a', 36), ('and', 19), ('his', 18), ('was', 13), ('with', 13), ('The', 10), ('is', 8), ('police', 7), ('that', 6)]


In [43]:
# 100 most frequent context words: now we are starting to see differences.
# We also see that many of the 100 most frequent context words only have counts of one.
print("100 most frequent contexts for some targets:\n")
print("election:\n", brown_context_counts["election"].most_common(100))
print("love:\n", brown_context_counts["love"].most_common(100))
print("car:\n", brown_context_counts["car"].most_common(100))

100 most frequent contexts for some targets:

election:
 [('the', 21), ('of', 11), ('to', 6), ('for', 4), ('was', 3), ('and', 3), ('his', 3), ('I', 3), ('on', 3), ('in', 3), ('an', 3), ('primary', 2), ('campaign', 2), ('last', 2), ('Presidential', 2), ('results', 2), ('November', 2), ('is', 2), ('April', 2), ('year', 2), ('produced', 1), ('laws', 1), ('general', 1), ('orderly', 1), ('8', 1), ('were', 1), ('investigation', 1), ('day', 1), ('told', 1), ('possible', 1), ('special', 1), ('might', 1), ('did', 1), ("you'll", 1), ('received', 1), ('they', 1), ('into', 1), ('bond', 1), ('The', 1), ('will', 1), ('Board', 1), ('Thursday', 1), ('His', 1), ('national', 1), ('close', 1), ('over', 1), ('procedures', 1), ('that', 1), ('inspired', 1), ('missed', 1), ('as', 1), ('falls', 1), ('law', 1), ('dates', 1), ('what', 1), ('plans', 1), ('may', 1)]
love:
 [('of', 36), ('and', 34), ('in', 26), ('for', 22), ('to', 21), ('the', 17), ('with', 14), ('I', 13), ('you', 10), ('is', 10), ('that', 7), ('m

In [44]:
# some ambiguous words
print("Some ambiguous words:")
print("bat:\n", brown_context_counts["bat"].most_common(100))
print("bank:\n", brown_context_counts["bank"].most_common(100))
print("bar:\n", brown_context_counts["bar"].most_common(100))
print("leave:\n", brown_context_counts["leave"].most_common(100))

Some ambiguous words:
bat:
 []
bank:
 [('the', 23), ('of', 9), ('a', 4), ('and', 4), ('in', 3), ('The', 2), ('local', 2), ('south', 2), ('which', 2), ('east', 2), ('west', 2), ('over', 1), ('accounts', 1), ('customers', 1), ('have', 1), ('That', 1), ('installed', 1), ('said', 1), ('is', 1), ('policy', 1), ('cloud', 1), ('that', 1), ('would', 1), ('for', 1), ('handling', 1), ('president', 1), ('to', 1), ('with', 1), ('officials', 1), ('before', 1), ('by', 1), ('loans', 1), ('left', 1), ('wrong', 1), ('far', 1), ('soft', 1), ('toward', 1), ('through', 1), ('river', 1), ('high', 1), ('outside', 1), ('big', 1), ('roll', 1)]
bar:
 [('the', 34), ('and', 10), ('locking', 10), ('a', 4), ('Af', 3), ('in', 3), ('while', 2), ('he', 2), ('to', 2), ('patent', 2), ('Would', 1), ('vehicles', 1), ('without', 1), ('cocktail', 1), ('which', 1), ('come', 1), ('at', 1), ('held', 1), ('A', 1), ('stock', 1), ('on', 1), ('should', 1), ('our', 1), ('is', 1), ('as', 1), ('who', 1), ('with', 1), ('so', 1), ('ab

## Selecting words to work with

We only keep words above a particular frequency threshold. 

In [45]:
# first determine the list of all words in Brown
# repeat: frequency threshold
frequency_threshold = 20

brown_wordlist = list(w for w in brown_wordcounts if brown_wordcounts[w] >= frequency_threshold)

In [46]:
# make a dictionary that maps each word to its index in the wordlist
brown_wordlist_lookup = { }

for index, word in enumerate(brown_wordlist):
    brown_wordlist_lookup[word] = index 

In [47]:
import numpy as np

# We need an array with enough space to hold 
# len(brown_wordlist) target words, and
# len(brown_wordlist) context words.
# We first initialize it to all zeros.
brown_count_matrix = np.zeros((len(brown_wordlist), len(brown_wordlist)))

In [48]:
# Now, let's do the context word counting with this matrix.

import string

for sentence in nltk.corpus.brown.sents():
    # remove punctuation.
    # at this point you could also remove stopwords
    # or iterate over sents_tagged() instead of sents()
    # to get parts of speech, and only retain
    # content words
    wordlist = [w for w in sentence if w.strip(string.punctuation) != ""]
    
    for targetindex, target in enumerate(wordlist):
        for contextword in each_contextword_1wordwindow(wordlist, targetindex):
            if brown_wordcounts[target] >= frequency_threshold and brown_wordcounts[contextword] >= frequency_threshold:
                # which cell in the matrix is this? 
                # look up both the target and the context word
                # in the ordered list of Brown words
                targetindex_matrix = brown_wordlist_lookup[target]
                contextindex_matrix = brown_wordlist_lookup[contextword]
                # and add a count of one for this cell in the matrix
                brown_count_matrix[targetindex_matrix][contextindex_matrix] += 1   


In [49]:
# We can again compute similarity in this space
said_index = brown_wordlist_lookup["said"]
wrote_index = brown_wordlist_lookup["wrote"]
cosine_sim( brown_count_matrix[said_index], brown_count_matrix[wrote_index])

0.8750717787803817

We now compute PMI again, making use of the fact that

$PMI(a, b) = \log \frac{P(a, b)}{P(a)P(b)} = \log\frac{P(b|a)}{P(b)}$

We re-implement PMI to run more quickly: 
numpy offers functions that apply an operation to a whole vector at once,
rather than one at a time. 
This is much quicker.

In [50]:
# And we can compute PMI again.
# numpy offers functions that
# apply an operation to a whole vector at once,
# rather than one at a time. 
# This is much quicker.

count_all = brown_count_matrix.sum()

# probability of contexts/columns:
# this is our P(b)
# This is a vector with a probability for each context
# sum(axis=0) is numpy's way of saying that we want to sum each column
col_totals = brown_count_matrix.sum(axis=0)
# avoid zeros, they get us in trouble later when we divide by p_c
col_totals[col_totals == 0] = 0.00001
# this is a vector where each row total is divided by the overall count
p_c = col_totals / count_all

# probability of context given target:
# this is our P(b|a)
# sum(axis=1) is numpy's way of saying that we want to sum each row:
# we divide each row by its row total, getting the probability of a context item
# within this target. 
# do do this, we flip the matrix on its side so that columns become rows,
# then do the division (otherwise numpy would do column-wise instead of row-wise division)
row_totals = brown_count_matrix.sum(axis=1).astype(float)
row_totals[row_totals == 0] = 0.00001
p_c_given_t = (brown_count_matrix.T / row_totals).T

# PMI: log( P(b|a) / P(b))
# we again divide a matrix by a vector
# this time we do want column-wise division
# so we don't have to flip the matrix
pct_divided_by_pc = p_c_given_t / p_c
# avoid doing log(0)by replacing 0 by a small number
pct_divided_by_pc[pct_divided_by_pc==0] = 0.00001

brown_pmi_matrix = np.log(pct_divided_by_pc)


In [51]:
# and computing similarity again
said_index = brown_wordlist_lookup["said"]
wrote_index = brown_wordlist_lookup["wrote"]
cosine_sim( brown_pmi_matrix[said_index], brown_pmi_matrix[wrote_index])

0.9244136837103797

In [52]:
# dimensionality reduction
# Principal component analysis
from sklearn.decomposition import PCA

pcaobj = PCA()
brown_pca_matrix = pcaobj.fit_transform(brown_pmi_matrix)

# and let's actually reduce dimensionality

keep_this_many_dimensions = 100

brown_pca_matrix = brown_pca_matrix[:, :keep_this_many_dimensions]

When we compute similarity again, the absolute value of the cosine similarity has changed a lot -- but that does not make a big difference. Absolute similarity values can vary widely across spaces, but they may still predict the same nearest neighbors

In [53]:
# and computing similarity again
said_index = brown_wordlist_lookup["said"]
wrote_index = brown_wordlist_lookup["wrote"]
cosine_sim( brown_pca_matrix[said_index], brown_pca_matrix[wrote_index])

0.43305558791233933